# 1. Importando os dados:

Abaixo, usamos pandas para importar os dados, mas logo transformamos o DataFrame em uma lista, para tentar nos adequar ao máximo ao conteúdo já ministrado.

In [1]:
import pandas as pd

dados = pd.read_json('brasileirao-2019.json')


campeonato = []

for i in dados.columns:
  campeonato.append(list(dados[i]))

# 2. Funções usadas:
Abaixo, criamos as funções que irão ser usadas ao longo do projeto:

In [12]:
def tecnicos(campeonato: list) -> list:
  """retorna a lista de técnicos que participaram do campeonato"""
  tec = sorted(
      list(
          set(
              [
                  tecnico
                  for rodada in campeonato
                  for partida in rodada
                  for tecnico in partida['coach'].values()
              ]
            )
          )
      )
  return tec



def times(campeonato: list) -> list:
  """retorna uma lista com os times que disputaram o campeonato em ordem alfabética"""
  t = sorted(
      list(
          set(
              [
                  time
                  for rodada in campeonato
                  for partida in rodada
                  for time in partida['clubs'].values()
              ]
            )
          )
      )
  return t



def times_por_tecnico(tecnico: str, campeonato: list) -> list:
  """Retorna uma lista com os times pelos quais o técnico atuou durante o campeonato"""
  return list(set([
    partida['clubs'][chave]
    for rodada in campeonato
    for partida in rodada
    for chave in ['home', 'away']
    if partida['coach'][chave] == tecnico
  ]))



def partidas_pelo_time(tecnico: str, time: str, campeonato: list) -> int:
    """recebe um técnico, um time e um campeonato,
    e retorna a quantidade de partidas disputadas pelo técnico no time"""
    return [
    partida['clubs'][chave]
    for rodada in campeonato
    for partida in rodada
    for chave in ['home', 'away']
    if (partida['coach'][chave] == tecnico
        and partida['clubs'][chave] == time)
  ].count(time)



def chave_tecnico(tecnico: str, partida: dict) -> str:
  """Recebe um tecnico e uma partida, e retorna a chave
  que indica se o tecnico jogou em casa ou fora de casa"""
  if tecnico not in partida['coach'].values():
    return ""
  return [ch for ch in partida['coach'].keys() if partida['coach'][ch] == tecnico][0]



def chave_time(time: str, partida: dict) -> str:
  """Recebe um time e uma partida, e retorna a chave
  que indica se o time jogou em casa ou fora de casa"""
  if time not in partida['clubs'].values():
    return ""
  return [ch for ch in partida['clubs'].keys() if partida['clubs'][ch] == time][0]



def tecnico_de(time: str, partida: dict) -> str:
  """Retorna o ténico do time na partida"""
  return "" if time not in partida['clubs'].values() else partida['coach'][chave_time(time, partida)]



def time_de(tecnico: str, partida: dict) -> str:
    """retorna o time treinado por um tecnico em uma partida"""
    return "" if tecnico not in partida['coach'].values() else partida['clubs'][chave_tecnico(tecnico, partida)]



def vencedor(partida: dict) -> str:
  """recebe uma partida e retorna o time vencedor"""
  if partida['goals']['home'] == partida['goals']['away']:
    return "empate"
  else:
    home = int(partida['goals']['home'])
    away = int(partida['goals']['away'])
    return partida['clubs']['home'] if home > away else partida['clubs']['away']



def pontos(time: str, campeonato: list) -> int:
  """retorna a quantidade de pontos de um time em um campeonato"""
  pontos = 0
  partidas = [
      partida
      for rodada in campeonato
      for partida in rodada
      if time in partida['clubs'].values()
    ]
  for partida in partidas:
    if vencedor(partida) == time:
      pontos += 3
    elif vencedor(partida) == 'empate':
      pontos += 1
  return pontos



def pontos_tecnico(tecnico: str, time: str, campeonato: list) -> int:
    """Retorna a quantidade de pontos que um técnico conseguiu para um dado time em um campeonato"""
    pontos = 0
    for rodada in campeonato:
        for partida in rodada:
            if time in partida['clubs'].values() and tecnico_de(time, partida) == tecnico:
                if vencedor(partida) == time:
                    pontos += 3
                elif vencedor(partida) == 'empate':
                    pontos += 1
    return pontos



def pontos_por_partida(tecnico: str, time: str, campeonato: list) -> float:
    """recebe um técnico, um time, e um capeonato, e retorna
    a quantidade de pontos por partida que este técnico fez pelo time"""
    return pontos_tecnico(tecnico, time, campeonato)/partidas_pelo_time(tecnico,time,campeonato)





In [13]:
time_mais_longevo_de("A. da Silva Braga", campeonato)

'Cruzeiro'

# 3. Pontos por Partida. 

Para tentar mensurar a eficiência de um técnico em um time, vamos calcular a razão entre a quantidade de pontos que o time obteve sob o comando do técnico e a quantidade de partidas disputadas pelo time sob o comando do mesmo técnico durante o campeonato.

Abaixo, temos uma relação de todos os técnicos que participaram do campeonato e, para cada um deles, os times em que atuaram, quantidade de pontos que conseguiram para o time, e a quantidade de pontos por partida que atingiram:

In [3]:
for tecnico in tecnicos(campeonato):
    print(tecnico+":")
    for time in times_por_tecnico(tecnico, campeonato):
        print(f" -> {time}:")
        print("    - Partidas disputadas:     ", partidas_pelo_time(tecnico, time, campeonato))
        print("    - Total de pontos obtidos: ", pontos_tecnico(tecnico, time, campeonato))        
        print("    - Pontos por partida:      ", pontos_por_partida(tecnico, time,campeonato))
        print()
    print()

A. Batista:
 -> Cruzeiro:
    - Partidas disputadas:      3
    - Total de pontos obtidos:  0
    - Pontos por partida:       0.0

 -> Ceará SC:
    - Partidas disputadas:      13
    - Total de pontos obtidos:  14
    - Pontos por partida:       1.0769230769230769


A. Fucks:
 -> CSA:
    - Partidas disputadas:      26
    - Total de pontos obtidos:  26
    - Pontos por partida:       1.0

 -> Ceará SC:
    - Partidas disputadas:      3
    - Total de pontos obtidos:  2
    - Pontos por partida:       0.6666666666666666


A. Lopes dos Santos:
 -> Palmeiras:
    - Partidas disputadas:      2
    - Total de pontos obtidos:  6
    - Pontos por partida:       3.0


A. Stival:
 -> São Paulo:
    - Partidas disputadas:      21
    - Total de pontos obtidos:  35
    - Pontos por partida:       1.6666666666666667


A. Valentim do Carmo Neto:
 -> Avaí:
    - Partidas disputadas:      15
    - Total de pontos obtidos:  13
    - Pontos por partida:       0.8666666666666667

 -> Botafogo:
    - P

Entretanto, há muitos técnicos que não tiveram tanta participação no campeonato. Alguns participaram de uma única partida em um único time. Alguns participaram de uma partida em um time e várias em outros. 

O fato é que nem todos os dados que temos são relevantes para nossa análise. Queremos saber se há alguma relação entre o tempo (em rodadas) que um técnico passou em um time e o desempenho do time em um campeonato. Alguns dados além de não serem relevantes, poluem nossa análise, dificultando uma visão mais clara da situação.

Com isso em mente, vamos considerar apenas técnicos que atuaram em mais de 10 partidas por um time, e vamos montar uma lista de tuplas. 

Cada tupla será composta de:
- Um Técnico;
- Um time pelo qual o técnico atuou no campeonato;
- Quantidade de Partidas do técnico pelo time;
- Pontos por partida do técnico no time;
- Quantidade de pontos do time no campeonato.

A lista de tuplas será então ordenada pelo total de partidas, ou seja, a longevidade do técnico no time.

In [20]:
tecnicos_times_relevantes = []
for tecnico in tecnicos(campeonato):
    for time in times_por_tecnico(tecnico, campeonato):
        if partidas_pelo_time(tecnico, time, campeonato) > 10:
            tecnicos_times_relevantes.append((tecnico,
                                              time,
                                              partidas_pelo_time(tecnico, time, campeonato),
                                              pontos_por_partida(tecnico, time, campeonato),
                                              pontos(time, campeonato)
                                             ))

tecnicos_times_relevantes = sorted(tecnicos_times_relevantes, key=lambda x: x[2])